In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir("../input/aaaaaaaaa"))
print(os.listdir("../input/ccccccccc"))
print(os.listdir('../input/dfdfdf/'))

In [ ]:
path_train_bus = "../input/aaaaaaaaa/business_train.json"
path_test_bus = "../input/aaaaaaaaa/business_test.json"
path_train_rev = "../input/ccccccccc/review_train.json"
path_test_rev = "../input/ccccccccc/review_test.json"
v2h = '../input/dfdfdf/df_hours'
v2c = '../input/dfdfdf/df_cat'

In [ ]:
bus = pd.read_json(path_train_bus, lines=True, orient="columns")
h = pd.read_csv(v2h)
c = pd.read_csv(v2c)

In [ ]:
import json as js
with open(path_train_rev, 'r') as f:
    lines = f.readlines()

raw = list(map(js.loads, lines))
train = pd.DataFrame(raw)

In [ ]:
bus = pd.read_json(path_train_bus, lines=True, orient="columns")

In [ ]:
bus.shape

In [ ]:
res = []
for i in range(154606):
    if bus['categories'][i] and 'Chinese' in bus['categories'][i] and 'Restaurants' in bus['categories'][i]:
        res.append(bus['business_id'][i])
len(res)

In [ ]:
real = train[train['business_id'].isin(res)]

In [ ]:
for i in range(100,200):
    if real['stars'].values[i]==1:
        print(i)

In [ ]:
i = 186
print(real['stars'].values[i])
real['text'].values[i]

In [ ]:
real.to_csv('real',index=False)

In [ ]:
comb = pd.concat([bus,h,c],axis=1)

In [ ]:
a = pd.merge(train, comb, how='left', on='business_id')

In [ ]:
a.head()

In [ ]:
attr = bus['attributes']
num_row = len(attr.values)
def two2one(a):
    if not a:
        return {}
    res = dict()
    for i in a.keys():
        if not '{' in a[i]:
            
            res[i] = a[i]
        else:
            temp = re.findall('\w+-\w+|\w+', a[i])
            temp_dict = {}
            for l in range(0,len(temp),2):
                temp_dict[temp[l]] = temp[l+1]
            res.update(two2one(temp_dict))
    return res
import re
columns = set()
for i in attr.values:
    k = two2one(i)
    columns.update(k.keys())
name = list(columns)
num_col = len(name)
table = [[1]*num_col for i in range(num_row)]
ind = list(attr.index)
for i in range(num_row):
    for j in range(num_col):
        table[i][j] = two2one(attr.values[i]).get(name[j], np.nan)
df = pd.DataFrame(table, columns=name,index=ind)


def cut(x):
    if isinstance(x, str):
        return x.strip('\'u')
    return x
for num,i in enumerate(name):
    df[i] = df[i].apply(cut)
    
def f(x):
    if x=='False':
        return 1==2
    if x=='True':
        return 1==1
    return x
for num,i in enumerate(name):
    df[i] = df[i].apply(f)


df.columns

In [ ]:
print(df.shape)
df.head()

In [ ]:
i = 7
print(df.columns[i])
df[df.columns[i]].value_counts(dropna=False)

In [ ]:
df.shape

In [ ]:
step1 = pd.concat([bus, df], axis=1)
step1 = step1.drop(['attributes'],axis=1)
print(len(step1.columns))

In [ ]:
step1.head()

In [ ]:
def helper_cat(x):
    if not x:
        return []
    temp = x.strip('\'').split(',')
    return [i.strip() for i in temp]
def deal_cat(cat):
    name = list(set([j for i in range(num_row) for j in helper_cat(cat.values[i])]))
    print(len(name))
    num_col = len(name)
    table = [[1]*num_col for i in range(num_row)]

    for i in range(num_row):
        for j in range(num_col):
            table[i][j] = True if name[j] in helper_cat(cat.values[i]) else False
    df = pd.DataFrame(table, columns=name)
    return df

In [ ]:
step1['categories'].value_counts()

In [ ]:
step2 = pd.concat([step1, deal_cat(step1['categories'])],axis=1)

In [ ]:
from datetime import datetime as dt
(dt.strptime('21:30','%H:%M')-dt.strptime('17:30','%H:%M')).total_seconds()/3600

In [ ]:
tm3=datetime.datetime.strptime("2018-01-16 23:44:55","%Y-%m-%d %H:%M:%S")

In [ ]:
step2['hours'].values[0]

In [ ]:
def prehelper(x):
    x1, x2 = x.split('-')
    return (dt.strptime('21:30','%H:%M')-dt.strptime('17:30','%H:%M')).total_seconds()/3600
prehelper('17:30-21:30')

In [ ]:
prehelper('7:0-1:0')

In [ ]:
def prehelper(x):
    x1, x2 = x.split('-')
    res = (dt.strptime(x2,'%H:%M')-dt.strptime(x1,'%H:%M')).total_seconds()/3600
    return res if res>0 else 24+res 
def helper(x):
    if not x:
        return [np.nan, np.nan, np.nan, np.nan]
    weekday = 0
    weekend = 0
    weekday_hour = 0
    weekend_hour = 0
    for i in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        if i in x.keys():
            if i in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
                weekday += 1
                weekday_hour += prehelper(x[i])
            else:
                weekend += 1
                weekend_hour += prehelper(x[i])
    return [weekday, weekend, weekday_hour, weekend_hour]

In [ ]:
helper(step2.hours.values[])

In [ ]:
def deal_hour(x):
    table = [[1]*4 for i in range(num_row)]
    for i in range(num_row):
        table[i] = helper(x.values[i])
    return pd.DataFrame(table, columns=['weekday', 'weekend', 'weekday_hour', 'weekend_hour'])

In [ ]:
df = deal_hour(step2.hours)


In [ ]:
j.strip() for i in a.values if i for j in i.strip('\'').split(',') if j

In [ ]:
rev = train_rev.__next__()
rev.columns

In [ ]:
rev.head()

In [ ]:
attr.values[9184]
def two2one(a):
    if not a:
        return {}
    res = dict()
    for i in a.keys():
        if not '{' in a[i]:
            
            res[i] = a[i]
        else:
            temp = re.findall('\w+-\w+|\w+', a[i])
            temp_dict = {}
            for l in range(0,len(temp),2):
                print(len(temp))
                temp_dict[temp[l]] = temp[l+1]
            res.update(two2one(temp_dict))
    return res
two2one(attr.values[9184])

In [ ]:
attr.values[9184]

In [ ]:
re.findall('\w+-\w+|\w+', 'a-a')

In [ ]:
import json as js
with open(path_train_bus, 'r') as f:
    lines = f.readlines()

raw = list(map(js.loads, lines))
raw = pd.DataFrame(raw)
num_row = raw.shape[0]

In [ ]:
def prehelper(x):
    x1, x2 = x.split('-')
    res = (dt.strptime(x2,'%H:%M')-dt.strptime(x1,'%H:%M')).total_seconds()/3600
    return res if res>0 else 24+res 
def helper(x):
    if not x:
        return [np.nan, np.nan, np.nan, np.nan]
    weekday = 0
    weekend = 0
    weekday_hour = 0
    weekend_hour = 0
    for i in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        if i in x.keys():
            if i in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
                weekday += 1
                weekday_hour += prehelper(x[i])
            else:
                weekend += 1
                weekend_hour += prehelper(x[i])
    return [weekday, weekend, weekday_hour, weekend_hour]


In [ ]:
def deal_hour(x):
    table = [[1]*4 for i in range(num_row)]
    for i in range(num_row):
        table[i] = helper(x.values[i])
    return pd.DataFrame(table, columns=['weekday', 'weekend', 'weekday_hour', 'weekend_hour'])

In [ ]:
df = deal_hour(raw.hours)
final = pd.concat([df,raw[['business_id']]],axis=1)

In [ ]:
final.to_csv('submit.csv', index=False)

In [ ]:
raw.columns

In [ ]:
pd.DataFrame?

In [ ]:
pd."DataFrame.value_counts

In [ ]:
str(a)

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
for i in bb.columns:
    bb[i] = lb.fit_transform(bb[i].astype(str))
import lightgbm as lgb
import pandas as pd
import pickle
from sklearn import model_selection
import matplotlib.pyplot as plt



def run_lgb(train_X, train_y, val_X, val_y):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "num_leaves": 30,
        "min_child_weight": 50,
        "learning_rate": 0.05,
        "bagging_fraction": 0.7,
        "feature_fraction": 0.7,
        "bagging_frequency": 5,
        "bagging_seed": 2018,
        "verbosity": -1
    }

    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100,
                      evals_result=evals_result)

    # pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return model, evals_result
#%%

train_X = bb
train_y = y

kf = model_selection.KFold(n_splits=5, random_state=2018, shuffle=True)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.iloc[dev_index, :], train_X.iloc[val_index, :]
    dev_y, val_y = train_y.iloc[dev_index], train_y.iloc[val_index]

    model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y)
#%%
fig, ax = plt.subplots(figsize=(12,10))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
